#**Problema 1 - Marco Amici**
Il progetto scelto è il numero 3 (CarbuRobot).
L'obiettivo è trovare una soluzione al problema che dallo stato iniziale, richiede di arrivare nello stato finale caratterizzato dai valori degli attributi (robot, hand, fuel, rooms) specificati nel goal state.

# Aima Python Utils


In [1]:
!pip install utils
import copy
import sys
from collections import deque
import math
from utils import *

![ -f /content/utils4e.py  ] &&  echo "exists" || wget https://raw.githubusercontent.com/aimacode/aima-python/master/utils4e.py
from utils4e import *

--2023-11-06 21:14:54--  https://raw.githubusercontent.com/aimacode/aima-python/master/utils4e.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23044 (23K) [text/plain]
Saving to: ‘utils4e.py’

utils4e.py          100%[===================>]  22.50K  --.-KB/s    in 0.002s  

2023-11-06 21:14:54 (10.4 MB/s) - ‘utils4e.py’ saved [23044/23044]



# Node Class

In [2]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        if  type(node.state) is dict:
            return self.state["robot"] < node.state["robot"]
        else:
            #otherwise is a tuple ( like Eight puzzle ) and we can easly check the greater.
            return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)

# Problem Class

In [3]:
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

# Algorithm

In [4]:
class SimpleProblemSolvingAgentProgram:
    """
    [Figure 3.1]
    Abstract framework for a problem-solving agent.
    """

    def __init__(self, initial_state=None):
        """State is an abstract representation of the state
        of the world, and seq is the list of actions required
        to get to a particular state from the initial state(root)."""
        self.state = initial_state
        self.seq = []

    def __call__(self, percept):
        """[Figure 3.1] Formulate a goal and problem, then
        search for a sequence of actions to solve it."""
        self.state = self.update_state(self.state, percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq = self.search(problem)
            if not self.seq:
                return None
        return self.seq.pop(0)

    def update_state(self, state, percept):
        raise NotImplementedError

    def formulate_goal(self, state):
        raise NotImplementedError

    def formulate_problem(self, state, goal):
        raise NotImplementedError

    def search(self, problem):
        raise NotImplementedError


# ______________________________________________________________________________
# Uninformed Search algorithms


def breadth_first_tree_search(problem, display=False):
    """
    [Figure 3.7]
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = deque([Node(problem.initial)])  # FIFO queue
    nodesCounter = 0
    while frontier:
        node = frontier.popleft()
        nodesCounter += 1
        if problem.goal_test(node.state):
            if display:
                print("Examined",nodesCounter,"nodes")
            return node
        frontier.extend(node.expand(problem))
    return None


def depth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = [Node(problem.initial)]  # Stack

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None


def depth_first_graph_search(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(problem.initial))]  # Stack

    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
    return None


def breadth_first_graph_search(problem):
    """[Figure 3.11]
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    """
    conta = 0
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = deque([node])
    explored = list()
    while frontier:
        node = frontier.popleft()
        conta = conta + 1
        if node.state not in explored:
            explored.append(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    print("Examined",conta,"nodes")
                    print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
                    return child
                frontier.append(child)
    return None


def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    conta = 0
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = list()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print("Examined",conta,"nodes")
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        if node.state not in explored:
            explored.append(node.state)
        for child in node.expand(problem):
            conta = conta + 1
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def astar_search_2(problem, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""

    node = Node(problem.initial)
    frontier = PriorityQueue('min', problem.astar_cost)
    frontier.append(node)
    while frontier:
        node = frontier.pop()
        #print(node)
        if problem.goal_test(node.state):
            if display:
                print(len(frontier), " paths remain in the frontier")
                print("Conta: ", conta, "Esaminati ", esaminati)
            return node
        for child in node.expand(problem):
            conta = conta + 1
            if child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if problem.astar_cost(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def astar_search_test(problem, h=None, display=False):
    esaminati=0
    node= Node(problem.initial)
    frontier= PriorityQueue('min', problem.astar_cost)
    frontier.append(node)
    while frontier:
        node = frontier.pop()
        esaminati = esaminati +1
        if problem.goal_test(node.state):
            if display:
                #print(node)
                #print(len(frontier), " path remain in the frontier")
                print("Esaminati ",esaminati)
            return node
        successors = node.expand(problem)
        conta = conta + len(successors)
        for child in successors:
            frontier.append(child)
    return None

def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost, display)

node_count = 0
def depth_limited_search(problem, limit=50, display=False):
    global node_count
    node_count = 0
    """[Figure 3.17]"""
    def recursive_dls(node, problem, limit):
        global node_count
        if problem.goal_test(node.state):
            if display:
                print("Examined ", node_count, "Nodes")
            return node
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                node_count +=1
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)


def iterative_deepening_search(problem, display=False):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth, display)
        if result != 'cutoff':
            return result


# ______________________________________________________________________________
# Bidirectional Search
# Pseudocode from https://webdocs.cs.ualberta.ca/%7Eholte/Publications/MM-AAAI2016.pdf

def bidirectional_search(problem):
    e = 0
    if isinstance(problem, GraphProblem):
        e = problem.find_min_edge()
    gF, gB = {Node(problem.initial): 0}, {Node(problem.goal): 0}
    openF, openB = [Node(problem.initial)], [Node(problem.goal)]
    closedF, closedB = [], []
    U = np.inf

    def extend(U, open_dir, open_other, g_dir, g_other, closed_dir):
        """Extend search in given direction"""
        n = find_key(C, open_dir, g_dir)

        open_dir.remove(n)
        closed_dir.append(n)

        for c in n.expand(problem):
            if c in open_dir or c in closed_dir:
                if g_dir[c] <= problem.path_cost(g_dir[n], n.state, None, c.state):
                    continue

                open_dir.remove(c)

            g_dir[c] = problem.path_cost(g_dir[n], n.state, None, c.state)
            open_dir.append(c)

            if c in open_other:
                U = min(U, g_dir[c] + g_other[c])

        return U, open_dir, closed_dir, g_dir

    def find_min(open_dir, g):
        """Finds minimum priority, g and f values in open_dir"""
        # pr_min_f isn't forward pr_min instead it's the f-value
        # of node with priority pr_min.
        pr_min, pr_min_f = np.inf, np.inf
        for n in open_dir:
            f = g[n] + problem.h(n)
            pr = max(f, 2 * g[n])
            pr_min = min(pr_min, pr)
            pr_min_f = min(pr_min_f, f)

        return pr_min, pr_min_f, min(g.values())

    def find_key(pr_min, open_dir, g):
        """Finds key in open_dir with value equal to pr_min
        and minimum g value."""
        m = np.inf
        node = Node(-1)
        for n in open_dir:
            pr = max(g[n] + problem.h(n), 2 * g[n])
            if pr == pr_min:
                if g[n] < m:
                    m = g[n]
                    node = n

        return node

    while openF and openB:
        pr_min_f, f_min_f, g_min_f = find_min(openF, gF)
        pr_min_b, f_min_b, g_min_b = find_min(openB, gB)
        C = min(pr_min_f, pr_min_b)

        if U <= max(C, f_min_f, f_min_b, g_min_f + g_min_b + e):
            return U

        if C == pr_min_f:
            # Extend forward
            U, openF, closedF, gF = extend(U, openF, openB, gF, gB, closedF)
        else:
            # Extend backward
            U, openB, closedB, gB = extend(U, openB, openF, gB, gF, closedB)

    return np.inf


# ______________________________________________________________________________
# Informed (Heuristic) Search


greedy_best_first_graph_search = best_first_graph_search


# Greedy best-first search is accomplished by specifying f(n) = h(n).


def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

# ______________________________________________________________________________




# Our problem

In [5]:
import copy

class robotLecture(Problem):
    def __init__(self, initial, goal):
        self.goal = goal
        Problem.__init__(self, initial, goal)
        self.corridor_size = 5

        self.objects = {
            "shoes": {"weight": 1},
            "desk": {"weight": 1},
            "candle": {"weight": 1},
            "bag": {"weight": 2},
            "station": {"weight": 0}
        }

    def actions(self, state):
        possible_actions = []

        current_room = state["rooms"][state["robot"]]

        # Azione get
        if state["hand"] is None and current_room != "null":
           for object_name, object_info in self.objects.items():
               object_weight = object_info["weight"]
               if object_name == current_room and object_weight <= state["fuel"]:
                  possible_actions.append("get_" + current_room)

        # Azione put
        if state["hand"] is not None and current_room == "null":
           object_name_in_hand = state["hand"]
           object_weight_in_hand = self.objects[object_name_in_hand]["weight"]
           if object_weight_in_hand <= state["fuel"]:
              possible_actions.append("put_" + object_name_in_hand)

        # Azione refill
        if "station" in current_room and state["hand"] is None:
            possible_actions.append("refill")

        # Azione per spostarsi a sinistra
        if state["robot"] > 0 and state["fuel"] >= 1:
           if state["hand"] is None or (state["hand"] in self.objects and self.objects[state["hand"]]["weight"] + 1 <= state["fuel"]):
              possible_actions.append("left")

        # Azione per spostarsi a destra
        if state["robot"] < self.corridor_size - 1 and state["fuel"] >= 1:
           if state["hand"] is None or (state["hand"] in self.objects and self.objects[state["hand"]]["weight"] + 1 <= state["fuel"]):
              possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        new_state = copy.deepcopy(state)

        action = action.split("_")

        if action[0] == "left":
          new_state["robot"] -= 1
          new_state["fuel"] -= 1
          if state["hand"] is not None:
            object_weight = self.objects[state["hand"]]["weight"]+1
            new_state["fuel"] -= object_weight
        if action[0] == "right":
          new_state["robot"] += 1
          new_state["fuel"] -= 1
          if state["hand"] is not None:
            object_weight = self.objects[state["hand"]]["weight"]+1
            new_state["fuel"] -= object_weight
        if action[0] == "get":
          object_name = action[1]
          object_weight = self.objects[object_name]["weight"]
          if state["fuel"] >= object_weight:
             new_state["hand"] = object_name
             new_state["rooms"][state["robot"]] = "null"
             new_state["fuel"] -= object_weight
        if action[0] == "put":
          object_name = action[1]
          object_weight = self.objects[object_name]["weight"]
          if state["fuel"] >= object_weight:
            new_state["hand"] = None
            new_state["rooms"][state["robot"]] = object_name
            new_state["fuel"] -= object_weight
        if action[0] == "refill":
          new_state["fuel"] += 10

        return new_state

    def goal_test(self, state):
        return (
            state["robot"] == self.goal["robot"] and
            state["hand"] == self.goal["hand"] and
            state["fuel"] >= self.goal["fuel"] and
            state["rooms"] == self.goal["rooms"]
        )



    def h1(self, node):
        current_state = node.state
        goal_state = self.goal

        misplaced_objects = 0
        for current_room, goal_room in zip(current_state["rooms"], goal_state["rooms"]):
            if current_room != "null" and set(current_room) != set(goal_room):
                misplaced_objects += 1

        return misplaced_objects

    def h2(self, node):
        current_state = node.state
        goal_state = self.goal

        misplaced_rooms = sum(1 for current_room, goal_room in zip(current_state["rooms"], goal_state["rooms"]) if set(current_room) != set(goal_room))

        return misplaced_rooms



# Esempio di utilizzo 1
initial_state = {
    "robot": 0,
    "hand": None,
    "fuel": 15,
    "rooms": ['station', 'candle', 'null', 'null','null']
}

goal_state = {
    "robot": 2,
    "hand": None,
    "fuel": 5,
    "rooms": ['station', 'null', 'candle', 'null', 'null']
}

# Esempio di utilizzo 2
initial_state1 = {
    "robot": 0,
    "hand": None,
    "fuel": 15,
    "rooms": ['station', 'candle', 'bag', 'null','null']
}

goal_state1 = {
    "robot": 3,
    "hand": None,
    "fuel": 5,
    "rooms": ['station', 'null', 'candle', 'bag', 'null']
}

# Esempio di utilizzo 3
initial_state2 = {
    "robot": 0,
    "hand": "bag",
    "fuel": 5,
    "rooms": ['null', 'station', 'desk', 'candle','null']
}

goal_state2 = {
    "robot": 4,
    "hand": "desk",
    "fuel": 5,
    "rooms": ['bag', 'station', 'null', 'null', 'candle']
}

problem = robotLecture(initial_state, goal_state)
problem1 = robotLecture(initial_state1, goal_state1)
problem2 = robotLecture(initial_state2, goal_state2)




## Controllo funzionamento di actions() e result()

In [6]:
# actions()

actions = problem.actions(initial_state)

# Stampare le azioni disponibili
print("Possible actions for initial state:")
for action in actions:
    print(action)

# result()

action = "right"

print("Apply "+action+" ...")
# Calcola il nuovo stato
new_state = problem.result(initial_state, action)

# Stampa il nuovo stato
print("New state after action: "+action , new_state)

Possible actions for initial state:
get_station
refill
right
Apply right ...
New state after action: right {'robot': 1, 'hand': None, 'fuel': 14, 'rooms': ['station', 'candle', 'null', 'null', 'null']}


## Ricerca della soluzione

In [7]:
import time

# Problema 1
# iterative deepening search
start_time = time.time()
s3_it = iterative_deepening_search(problem, display=True)
end_time = time.time()
print("Iterative Deepening Search Solution:", s3_it.solution())
print("Iterative Deepening Search Time:", end_time - start_time, "seconds\n\n")

# breadth-first tree search
start_time = time.time()
s3_bf = breadth_first_tree_search(problem, display=True)
end_time = time.time()
print("Breadth-First Tree Search Solution:", s3_bf.solution())
print("Breadth-First Tree Search Time:", end_time - start_time, "seconds\n\n")

# uniform cost search
start_time = time.time()
sol2 = uniform_cost_search(problem, display=True)
end_time = time.time()
print("Uniform Cost Search Solution:", sol2.solution())
print("Uniform Cost Search Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h1
start_time = time.time()
s3_h1 = astar_search(problem, problem.h1, display=True)
end_time = time.time()
print("Heuristic Search with h1 Solution:", s3_h1.solution())
print("Heuristic Search with h1 Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h2
start_time = time.time()
s3_h2 = astar_search(problem, problem.h2, display=True)
end_time = time.time()
print("Heuristic Search with h2 Solution:", s3_h2.solution())
print("Heuristic Search with h2 Time:", end_time - start_time, "seconds\n\n")

###########################################################################
# Problema 2
# iterative deepening search
start_time = time.time()
s3_it1 = iterative_deepening_search(problem1, display=True)
end_time = time.time()
print("Iterative Deepening Search Solution:", s3_it1.solution())
print("Iterative Deepening Search Time:", end_time - start_time, "seconds\n\n")

# breadth-first tree search
start_time = time.time()
s3_bf1 = breadth_first_tree_search(problem1, display=True)
end_time = time.time()
print("Breadth-First Tree Search Solution:", s3_bf1.solution())
print("Breadth-First Tree Search Time:", end_time - start_time, "seconds\n\n")

# uniform cost search
start_time = time.time()
sol2_u = uniform_cost_search(problem1, display=True)
end_time = time.time()
print("Uniform Cost Search Solution:", sol2_u.solution())
print("Uniform Cost Search Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h1
start_time = time.time()
s3_he = astar_search(problem1, problem1.h1, display=True)
end_time = time.time()
print("Heuristic Search with h1 Solution:", s3_he.solution())
print("Heuristic Search with h1 Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h2
start_time = time.time()
s3_he1 = astar_search(problem1, problem1.h2, display=True)
end_time = time.time()
print("Heuristic Search with h2 Solution:", s3_he1.solution())
print("Heuristic Search with h2 Time:", end_time - start_time, "seconds\n\n")

###############################################################################
#Problema 3
# iterative deepening search
start_time = time.time()
s3_it2 = iterative_deepening_search(problem2, display=True)
end_time = time.time()
print("Iterative Deepening Search Solution:", s3_it2.solution())
print("Iterative Deepening Search Time:", end_time - start_time, "seconds\n\n")

# breadth-first tree search
start_time = time.time()
s3_bf2 = breadth_first_tree_search(problem2, display=True)
end_time = time.time()
print("Breadth-First Tree Search Solution:", s3_bf2.solution())
print("Breadth-First Tree Search Time:", end_time - start_time, "seconds\n\n")

# uniform cost search
start_time = time.time()
sol2_u2 = uniform_cost_search(problem2, display=True)
end_time = time.time()
print("Uniform Cost Search Solution:", sol2_u2.solution())
print("Uniform Cost Search Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h1
start_time = time.time()
s3_he2 = astar_search(problem2, problem2.h1, display=True)
end_time = time.time()
print("Heuristic Search with h1 Solution:", s3_he2.solution())
print("Heuristic Search with h1 Time:", end_time - start_time, "seconds\n\n")

# heuristic search with h2
start_time = time.time()
s3_he3 = astar_search(problem2, problem2.h2, display=True)
end_time = time.time()
print("Heuristic Search with h2 Solution:", s3_he3.solution())
print("Heuristic Search with h2 Time:", end_time - start_time, "seconds\n\n")








Examined  64 Nodes
Iterative Deepening Search Solution: ['right', 'get_candle', 'right', 'put_candle']
Iterative Deepening Search Time: 0.0026395320892333984 seconds


Examined 72 nodes
Breadth-First Tree Search Solution: ['right', 'get_candle', 'right', 'put_candle']
Breadth-First Tree Search Time: 0.002409219741821289 seconds


Examined 109 nodes
43 paths have been expanded and 40 paths remain in the frontier
Uniform Cost Search Solution: ['right', 'get_candle', 'right', 'put_candle']
Uniform Cost Search Time: 0.0020813941955566406 seconds


Examined 74 nodes
29 paths have been expanded and 31 paths remain in the frontier
Heuristic Search with h1 Solution: ['right', 'get_candle', 'right', 'put_candle']
Heuristic Search with h1 Time: 0.005043745040893555 seconds


Examined 32 nodes
12 paths have been expanded and 15 paths remain in the frontier
Heuristic Search with h2 Solution: ['right', 'get_candle', 'right', 'put_candle']
Heuristic Search with h2 Time: 0.0028061866760253906 seconds